# Get Histogram

Import all modules:

In [ ]:
import drp_template.input_output as io
from drp_template.image import plot_histogram as plt_hist
from drp_template.image import save_figure2

Set parameters for import uint16.raw:

In [ ]:
file_path = '../examples/data/raw_100cube.raw'
dimensions = {'nz': 100, 'ny': 100, 'nx': 100}
data = io.import_model(file_path=file_path, dtype='uint16', dimensions=dimensions)

Plot histogram for uint16.raw and save it accordingly:

In [ ]:
fig, ax = plt_hist(data, paramsfile='raw_100cube.json', title=None, log_scale='y', dark_mode=False)
save_figure2(fig, filename='histogram_raw_100cube')